### Notebook to create BLOCK-T187 in Elevation with closed loop

Created on: 2024-10-09

Author: Guillem Megias

In [ ]:
from lsst.ts.observing import ObservingBlock, ObservingScript 
from lsst.ts.aos.analysis import build_configuration_schema
import os

In [ ]:
current_path = os.getcwd()
block_number = 'T187'
name = "BLOCK-T187"
program = "BLOCK-T187"
reason = "SITCOM-1489"
note = "LUT_elevation_closed_loop"
constraints = []

### Define configuration schema

In [ ]:
# Define the configurable properties that we will use in the configuration schema
properties = {
    "run_filter": {
        "description": "ComCam filter to use.",
        "type": "string",
        "default": "r_03"
    },
    "run_azimuth": {
        "description": "Azimuth to use for the elevation sweep.",
        "type": "number",
        "default": 20
    },
    "run_dofs": {
        "description": "Degrees of freedom to use for the elevation sweep.",
        "type": "array",
        "items": {
            "type": "integer",
            "minimum": 0,
            "maximum": 49,
        },
        "default": list(range(0, 50))
    },
    "run_maxiter": {
        "description": "Maximum number of iterations for the elevation sweep.",
        "type": "integer",
        "default": 5
    }
}

# Build the configuration schema
configuration_schema = build_configuration_schema(block_number, properties)
print(configuration_schema)

### Define scripts

In [ ]:
comcam_closedloop_script = ObservingScript(
    name="maintel/close_loop_comcam.py",
    standard=True,
    parameters= dict(
        filter="$run_filter",
        program="$program",
        reason=reason,
        note=note,
        exposure_time=30.0,
        max_iter="$run_maxiter",
        used_dofs="$run_dofs",
    )
)

stop_tracking_script = ObservingScript(
    name="maintel/stop_tracking.py",
    standard=True,
    parameters = dict()
)

### Define scripts and block

In [ ]:
elevation_positions = [20, 30, 40, 50, 60, 70, 80, 70, 60, 50, 40, 30, 20]
scripts = []

for elevation in elevation_positions:
    track_target_script = ObservingScript(
        name="maintel/track_target.py",
        standard=True,
        parameters = dict(
            track_azel = dict(
                az = '$run_azimuth',
                el = elevation
            )
        )
    )

    scripts.append(track_target_script)
    scripts.append(comcam_closedloop_script)
scripts.append(stop_tracking_script)

In [ ]:
block = ObservingBlock(
    name = program,
    program = program,
    configuration_schema=configuration_schema,
    scripts = scripts,
)

### Save configurable block

In [ ]:
block.model_dump_json(indent=2)

output_file_path = f'{current_path}/aos/ts_config_ocs/Scheduler/observing_blocks_maintel/AOS/LUTs/{program}.json'

with open(output_file_path, 'w') as file:
    file.write(block.model_dump_json(indent=2))